In [ ]:
%pylab notebook

import scipy, scipy.linalg, scipy.signal
import itertools
import tidynamics

In [ ]:
plt.rcParams['figure.figsize'] = (10,6)

In [ ]:
def mycor(a, b, N_max=None):                      
    N = len(a)                                             
    if N_max is None:
        N_max = N//10
    res = scipy.signal.fftconvolve(a[:], b[::-1])[N-1:] / (N-np.arange(N))
    return res[:N_max]


In [ ]:
Γ = np.array([[400, 50, -200],[50, 600, 250], [-200, 250, 4000]])

Γ = np.array([[  395.24876886,    49.64205652,  -211.47449726],
       [   53.02416734,   568.67494294,   288.95198073],
       [ -214.96374295,   273.39955903,  4100.94214944]])

Γ = np.eye(3)*10
Γ[0,2] = Γ[2, 0] = 3

Mdiag = np.array([14745, 14745, 107233])

Mdiag = np.array([100, 100, 100])

oosqrt_M = np.diagflat(1/np.sqrt(Mdiag))

γ = oosqrt_M @ Γ @ oosqrt_M

In [ ]:
w, v = np.linalg.eig(-γ)
vinv = np.linalg.inv(v)

In [ ]:
print(w, v)

In [ ]:
dt = 0.05

a = v @ np.diagflat(np.exp(w*dt)) @ vinv
sqrt_a = v @ np.sqrt(np.diagflat(np.exp(w*dt))) @ vinv
one_minus_a = v @ (np.eye(3) - np.diagflat(np.exp(w*dt))) @ vinv
sqrt_one_minus_a = v @ np.sqrt((np.eye(3) - np.diagflat(np.exp(w*dt)))) @ vinv

In [ ]:
print(a)
print(sqrt_a)

assert np.allclose(a, sqrt_a @ sqrt_a)

In [ ]:
vel = np.zeros(3)

F = np.array([0, 0, 0])

FF = F*dt/np.sqrt(Mdiag)

data = []
v_data = []
for i in range(32768*2):
    for j in range(10):
        vel = sqrt_a @ vel + sqrt_one_minus_a @ np.random.normal(size=3)
        vel += FF
        vel = sqrt_a @ vel + sqrt_one_minus_a @ np.random.normal(size=3)

    data.append(vel)
    v_data.append(oosqrt_M @ vel)

data = np.array(data)
v_data = np.array(v_data)

In [ ]:
def theo(t):
    return v @ np.diagflat(np.exp(w*t)) @ vinv

N_plot = int(100/dt)

T = dt*np.arange(N_plot)*10

tdata = np.array([theo(t) for t in T])


In [ ]:
plt.figure()

ax = [plt.subplot(3, 3, 1+3*i+j) for i, j in itertools.product(range(3), range(3))]
for i, j in itertools.product(range(3), range(3)):
    cross_mean = 0 if i==j else (data[:,i]*data[:,j]).mean()
    ax[3*i+j].plot(T, mycor(data[:,i], data[:,j], N_max=N_plot)-cross_mean)
    ax[3*i+j].plot(T, tdata[:,i,j])
    ax[3*i+j].plot(T, tidynamics.core.correlation_1d(data[:,i], data[:,j])[-len(data[:,i]):][:N_plot]-cross_mean)


In [ ]:
tdata.shape

In [ ]:
plt.figure()
vrange = np.linspace(-3/10, 3/10, 101)

for i in range(3):
    plt.subplot(3, 1, i+1)
    dum = hist(v_data[:,i], bins=32, normed=True, histtype='step', lw=2)

    plt.plot(vrange, np.sqrt(Mdiag[i]/(2*np.pi))*np.exp(-Mdiag[i]*vrange**2/2))


#[plt.axvline(m, color=l.get_color(), lw=2) for m, l in zip(data.mean(axis=0)/np.sqrt(Mdiag), lines)];

#[plt.axvline(vstall, color=l.get_color(), lw=2, ls='--') for vstall, l in zip(np.linalg.inv(Γ) @ F, lines)];

In [ ]:
Γ_inv = np.linalg.inv(Γ)
v_stall = Γ_inv @ F

print('Radius from stall velocity')
print(np.sqrt(v_stall[0]**2+v_stall[1]**2)/v_stall[2])
print('Radius from friction matrix')
print(np.sqrt(Γ_inv.T[0,0]**2+Γ_inv.T[0,1]**2)/Γ_inv.T[0,2])
v_mean = data.mean(axis=0)/np.sqrt(Mdiag)
print('Radius from mean velocity')
print(np.sqrt(v_mean[0]**2+v_mean[1]**2)/v_mean[2])

In [ ]:
Γ

In [ ]:
np.sqrt(γ[0,0]**2+γ[0,1]**2)/γ[0,2]

In [ ]:
np.sqrt(data[:,0].mean()**2+data[:,1].mean()**2)/data[:,2].mean()

In [ ]:
Γ_inv

In [ ]:
pos = scipy.integrate.cumtrapz(v_data, axis=0)[::32,:]*10*dt

In [ ]:
cd = tidynamics.cross_displacement(pos)

In [ ]:
plt.figure()

pos_time = np.arange(pos.shape[0])*32*10*dt

ax = [plt.subplot(3, 3, 1+3*i+j) for i, j in itertools.product(range(3), range(3))]

for i, j in itertools.product(range(3), range(3)):
    plt.subplot(3, 3, 1+3*i+j)
    plt.plot(pos_time[:400], cd[i][j][:400])
    if i==j:
        plt.plot(pos_time[:400], 2*Γ_inv[i, i]*pos_time[:400])
    else:
        #plt.plot(T, T*scipy.integrate.cumtrapz((tdata[:,i,j]+tdata[:,j,i])*(1-1/(T+1e-6)), initial=0, dx=dt*10)*(oosqrt_M[i,i]*oosqrt_M[j,j]))
        plt.plot(T, t_msd(i, j))
    plt.xlim(0, 2000)


In [ ]:
def t_msd(i, j):
    def local_integral(t_idx):
        t = T[t_idx]
        cross_mean = 0 if i==j else tdata[0,i,j]
        return t*scipy.integrate.trapz((tdata[:t_idx,i,j]+tdata[:t_idx,j,i]-2*cross_mean)*(1-T[:t_idx]/(t+1e-6)), dx=10*dt)
    return np.array([local_integral(i) for i in range(len(T))])*(oosqrt_M[i,i]*oosqrt_M[j,j])


In [ ]:
oosqrt_M

In [ ]:
i, j